In [1]:
FILE_PATH = '/home/matt/repos/ohia.ai/data'

In [3]:
model_name = 'mobilenetv1'
seed = 1
batch_size = 32
n_hidden = 0
dropout = 0.0
filtered = False
augmentation = False
n_thread = 32
gpu = 0
save_model = False

In [4]:
import os, re, glob, json
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=str(gpu)

import numpy as np
import pandas as pd

from keras import callbacks
from sklearn.model_selection import train_test_split

from ohia.encoders import FastLabelEncoder

from ohia.utils import PlantNetGenerator, make_dir

Using TensorFlow backend.


In [11]:
# set up paths
IMAGE_DIR = 'filtered_plantnet'
model_path = (
    f'{FILE_PATH}/models/plantnet_{model_name}' +
    f'_seed-{seed}' +
    f'_batch_size-{batch_size}' +
    f'_n_hidden-{n_hidden}' +
    f'_dropout-{dropout:0.2f}' +
    ('_filtered' if filtered else '') +
    ('_augmentation' if augmentation else '')
)
print(model_path)
make_dir(model_path)

/home/matt/repos/ohia.ai/data/models/plantnet_mobilenetv1_seed-1_batch_size-32_n_hidden-0_dropout-0.00


In [12]:
model.load_weights(weight_file)# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]

# encode label names with ids
fle = FastLabelEncoder()
label_ids = fle.fit_transform(full_label_list)

# save id2label map
id2label = {int(fle.transform([label])):label for label in np.unique(full_label_list)}
with open(f'{model_path}/plantnet_classes.json', 'w') as fp:
    json.dump(id2label, fp)

In [13]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, label_ids, test_size=0.1, random_state=seed)

# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(
    train_files, train_ids, n_classes,
    batch_size=batch_size,
    augment=augmentation
)
valid_generator = PlantNetGenerator(
    valid_files, valid_ids, n_classes,
    batch_size=batch_size,
    augment=augmentation,
    shuffle=False
)

In [14]:
model.load_weights(weight_file)# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.25,
        patience=2,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{model_path}/weights' + '_{epoch:02d}.h5',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
    ),
]

In [15]:
def get_model(model_name, n_classes, n_hidden, dropout):

    from keras import layers, Model
    from keras.optimizers import Adam
    from keras.applications.mobilenet import MobileNet
    from keras.applications.mobilenetv2 import MobileNetV2
    from keras.applications.nasnet import NASNetMobile
    from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy

    # load pretrained ImageNet model
    if model_name == 'mobilenetv1':
        base_model = MobileNet(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )        
    elif model_name == 'mobilenetv2':
        base_model = MobileNetV2(
            input_shape=(224,224,3),
            alpha=1.4,
            weights='imagenet',
            include_top=False
        )
    elif model_name == 'nasnetmobile':
        base_model = NASNetMobile(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )
    else:
        assert ValueError(
            f'model_name parameter must be one of the following'
            ' "mobilenetv1",'
            ' "mobilenetv2",'
            ' "nasnetmobile"'
        )

#     # set freeze all layers
#     for layer in base_model.layers:
#         if re.findall('_\\d_', layer.name):
#             print(layer.name)
#             layer.trainable = False
        
    # map ImageNet features to plants
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    if dropout>0:
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    # compile the model
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(
        optimizer=Adam(lr=0.0001),
        loss='categorical_crossentropy',
        metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
    )

    return model

In [16]:
# train model 
model = get_model(model_name, n_classes, n_hidden, dropout)
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=n_thread,
    epochs=100,    
)

Epoch 1/100
2463/2463 [==============================] - 468s 190ms/step - loss: 3.7791 - top_1_accuracy: 0.2773 - top_3_accuracy: 0.4102 - top_5_accuracy: 0.4724 - val_loss: 2.6635 - val_top_1_accuracy: 0.4328 - val_top_3_accuracy: 0.5964 - val_top_5_accuracy: 0.6605
Epoch 2/100
2463/2463 [==============================] - 440s 179ms/step - loss: 2.1221 - top_1_accuracy: 0.5420 - top_3_accuracy: 0.6967 - top_5_accuracy: 0.7554 - val_loss: 2.0563 - val_top_1_accuracy: 0.5442 - val_top_3_accuracy: 0.7004 - val_top_5_accuracy: 0.7569
Epoch 3/100
2463/2463 [==============================] - 443s 180ms/step - loss: 1.5106 - top_1_accuracy: 0.6629 - top_3_accuracy: 0.7988 - top_5_accuracy: 0.8457 - val_loss: 1.8474 - val_top_1_accuracy: 0.5791 - val_top_3_accuracy: 0.7343 - val_top_5_accuracy: 0.7916
Epoch 4/100
2463/2463 [==============================] - 440s 179ms/step - loss: 1.1079 - top_1_accuracy: 0.7507 - top_3_accuracy: 0.8673 - top_5_accuracy: 0.9038 - val_loss: 1.7359 - val_top_1

In [17]:
# save results
results = pd.DataFrame(model.history.history)
results.to_csv(f'{model_path}/results.csv', index=False)

In [18]:
# print best results
print(results.iloc[results.val_top_3_accuracy.values.argmax()])

val_loss              1.595575
val_top_1_accuracy    0.665522
val_top_3_accuracy    0.795788
val_top_5_accuracy    0.842033
loss                  0.017637
top_1_accuracy        0.999784
top_3_accuracy        1.000000
top_5_accuracy        1.000000
lr                    0.000002
Name: 18, dtype: float64
